In [1]:
from transformers import AutoTokenizer, AutoModel

In [2]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path='bert-base-chinese') 
tokenizer

BertTokenizerFast(name_or_path='bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [3]:
text = '月光的[UNK][PAD][CLS]新希望[EOS]<eop>'

# 未添加新tokens前的编码效果
print(tokenizer.convert_tokens_to_ids(['月光', '希望', '[EOS]', '<eop>']))
print(tokenizer.tokenize(text))
print(len(tokenizer.get_vocab()))

[100, 100, 100, 100]
['月', '光', '的', '[UNK]', '[PAD]', '[CLS]', '新', '希', '望', '[', '[UNK]', ']', '<', 'e', '##op', '>']
21128


In [4]:
# Add a list of new tokens to the tokenizer class.
tokenizer.add_tokens(new_tokens=['月光', '希望'])

# Add a dictionary of special tokens (eos, pad, cls, etc.) to the encoder and link them to class attributes.
# If special tokens are NOT in the vocabulary, they are added to it (indexed starting from the last index of the current vocabulary).
print(tokenizer.eos_token, tokenizer.eos_token_id)
tokenizer.add_special_tokens(
    # Keys should be in the list of predefined special attributes: [`bos_token`, `eos_token`, `unk_token`, `sep_token`, `pad_token`, `cls_token`, `mask_token`, `additional_special_tokens`].
    special_tokens_dict={'eos_token': '[EOS]', 'additional_special_tokens': ["<eop>", "<eod>"]})
print(tokenizer.eos_token, tokenizer.eos_token_id)
print(len(tokenizer.get_vocab()))
print(tokenizer.special_tokens_map)
print(tokenizer.additional_special_tokens, tokenizer.additional_special_tokens_ids)

None None
[EOS] 21130
21133
{'eos_token': '[EOS]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]', 'additional_special_tokens': ['<eop>', '<eod>']}
['<eop>', '<eod>'] [21131, 21132]


In [5]:
# 添加新tokens后的编码效果
print(tokenizer.convert_tokens_to_ids(['月光', '希望', '[EOS]', '<eop>']))
print(tokenizer.tokenize(text))
print(len(tokenizer.get_vocab()))

[21128, 21129, 21130, 21131]
[' 月  光 ', '的', '[UNK]', '[PAD]', '[CLS]', '新', ' 希  望 ', '[EOS]', '<eop>']
21133


In [6]:
# 保存分词器(包括新添加的tokens)
tokenizer.save_pretrained("../extra_dataset/save_tokenizer/")

('../extra_dataset/save_tokenizer/tokenizer_config.json',
 '../extra_dataset/save_tokenizer/special_tokens_map.json',
 '../extra_dataset/save_tokenizer/vocab.txt',
 '../extra_dataset/save_tokenizer/added_tokens.json',
 '../extra_dataset/save_tokenizer/tokenizer.json')

In [7]:
# 从本地重新加载
tokenizer1 = AutoTokenizer.from_pretrained("../extra_dataset/save_tokenizer/")
tokenizer1

BertTokenizerFast(name_or_path='../extra_dataset/save_tokenizer/', vocab_size=21128, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '[EOS]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]', 'additional_special_tokens': ['<eop>', '<eod>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	21128: AddedToken("月光", rstrip=False, lstri

In [8]:
print(tokenizer1.convert_tokens_to_ids(['月光', '希望', '[EOS]', '<eop>']))
print(tokenizer1.tokenize(text))
print(len(tokenizer1.get_vocab()))

[21128, 21129, 21130, 21131]
[' 月  光 ', '的', '[UNK]', '[PAD]', '[CLS]', '新', ' 希  望 ', '[EOS]', '<eop>']
21133


In [9]:
model = AutoModel.from_pretrained('bert-base-chinese')

In [10]:
model(**tokenizer(text, return_tensors='pt'))  # 报错

IndexError: index out of range in self

In [11]:
model.embeddings

BertEmbeddings(
  (word_embeddings): Embedding(21128, 768, padding_idx=0)
  (position_embeddings): Embedding(512, 768)
  (token_type_embeddings): Embedding(2, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [12]:
# word_embedding维度为:21128 * 768(but此时tokenizer大小为:21133)
model.get_input_embeddings()

Embedding(21128, 768, padding_idx=0)

In [13]:
# Resizes input token embeddings matrix of the model if `new_num_tokens != config.vocab_size`.
model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(21133, 768, padding_idx=0)

In [14]:
model(**tokenizer(text, return_tensors='pt'))

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.3367,  0.7263,  0.5332,  ...,  0.3420, -0.5114, -0.0175],
         [-0.3379,  0.1868,  0.1348,  ...,  0.0349, -0.7228, -0.4014],
         [-0.5350,  0.4015,  0.6540,  ..., -0.2214, -0.3078,  0.0057],
         ...,
         [-0.5030,  0.4545,  0.6126,  ...,  0.5991, -0.3347, -0.3338],
         [-0.4078,  0.8154,  0.7629,  ..., -0.2217, -0.5943,  0.0385],
         [-0.3751,  0.7748,  0.1946,  ...,  0.6225, -0.9416,  0.1327]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 0.9987,  0.9997,  0.9953,  0.9943,  0.6537, -0.1105, -0.8766, -0.9217,
          0.9982, -0.9998,  1.0000,  0.9986, -0.7282, -0.9041,  0.9998, -0.9984,
         -0.6943,  0.9976,  0.9933,  0.0379,  0.9960, -1.0000, -0.9166, -0.8110,
         -0.5432,  0.9988,  0.8344, -0.9641, -0.9999,  0.9990,  0.9801,  0.9995,
          0.8950, -0.9999, -0.9996,  0.7325,  0.3060,  0.9951, -0.3898, -0.9223,
         -0.9804, -0.8721, -0.15